# Model Berbasis Expert System, Fuzzy Logic, dan Machine Learning untuk Menentukan Risiko Gagal Bayar Kredit

IMPORT LIBRARY

Meng-import semua library yang dibutuhkan, seperti numpy, pandas, xgboost, dll.

In [ ]:
# MAKE SURE TO RESTART SESSION BEFORE RUNNING THE CODE TO AVOID LIBRARY ERROR (COLAB)
# MAKE SURE DATASET IS UPLOADED IN COLAB

# TO RUN IN GOOGLE COLAB
!pip install pandas numpy scikit-learn imbalanced-learn xgboost shap matplotlib joblib --quiet
!pip install git+https://github.com/omadson/fuzzy-c-means.git --quiet

In [4]:
# IMPORT LIBRARY
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from fcmeans import FCM
import shap
import matplotlib.pyplot as plt
import joblib
import warnings
warnings.filterwarnings('ignore')

# LOAD DATA
df = pd.read_csv("credit_risk_dataset.csv") # Make sure dataset is uploaded in Colab

ENCODING

Mengubah status riwayat kredit macet dari huruf ('Y' = ya, 'N' = tidak) menjadi angka (1 = ya, 0 = tidak).

FEATURE ENGINEERING

* Membuat fitur baru bernama Aset Tetap, diambil langsung dari kolom Rumah yang telah diubah menjadi 0/1.
* Menentukan apakah seseorang punya pekerjaan yang stabil (lama kerja ≥ 5 tahun)
1 = stabil, 0 = tidak stabil
* Menyalin kolom rasio pinjaman terhadap penghasilan agar namanya lebih mudah dipahami.

DROP NaN

Menghapus semua baris yang memiliki nilai kosong (NaN) karena metode SMOTE (untuk balancing data) dan model ML tidak bisa jalan kalau masih ada nilai kosong.


In [5]:
# ENCODING CATEGORICAL
df['cb_person_default_on_file'] = df['cb_person_default_on_file'].map({'Y': 1, 'N': 0})
df['person_home_ownership'] = df['person_home_ownership'].map(lambda x: 1 if x == 'OWN' else 0)

# FEATURE ENGINEERING
df['has_fixed_asset'] = df['person_home_ownership']
df['stable_job'] = df['person_emp_length'].apply(lambda x: 1 if x >= 5 else 0)
df['loan_to_income_ratio'] = df['loan_percent_income']

# DROP NaN (Prevent SMOTE Error)
df = df.dropna()

SISTEM PAKAR (CERTAINTY FACTOR)

Untuk meniru penilaian manusia dalam menilai apakah nasabah berpotensi gagal bayar atau tidak berdasarkan aturan pakar.
*   cf_def: keyakinan bahwa nasabah akan default (gagal bayar)
*   cf_non: keyakinan bahwa nasabah tidak akan default

Jika Nasabah dianggap berisiko (CF default):
1. Penghasilan kecil dan baru kerja
2. Pernah macet dan tidak punya rumah
3. Masih muda dan penghasilan kecil

Jika Nasabah dianggap aman (CF non-default):
1. Lama kerja dan tidak pernah macet
2. Pendapatan tinggi dan pinjaman kecil
3.  Punya rumah dan tidak macet

CF_diff → selisih keyakinan default dan tidak default

hasil_pakar → hasil klasifikasi pakar (0 = aman, 1 = gagal bayar)

In [6]:
# EXPERT SYSTEM: CERTAINTY FACTOR
def rule_cf(row):
    cf_def, cf_non = [], []
    if row['person_income'] < 4000 and row['person_emp_length'] < 1: cf_def.append(0.85)
    if row['cb_person_default_on_file'] == 1 and row['has_fixed_asset'] == 0: cf_def.append(0.9)
    if row['person_age'] < 25 and row['person_income'] < 3000: cf_def.append(0.6)

    if row['person_emp_length'] >= 5 and row['cb_person_default_on_file'] == 0: cf_non.append(0.8)
    if row['person_income'] > 8000 and row['loan_percent_income'] < 0.3: cf_non.append(0.7)
    if row['has_fixed_asset'] == 1 and row['cb_person_default_on_file'] == 0: cf_non.append(0.75)

    def combine(lst):
        if not lst: return 0.0
        cf = lst[0]
        for val in lst[1:]:
            cf += val * (1 - cf)
        return round(cf, 4)

    return combine(cf_def), combine(cf_non)

df[['CF_default', 'CF_nondefault']] = df.apply(rule_cf, axis=1, result_type='expand')
df['CF_diff'] = df['CF_default'] - df['CF_nondefault']
df['expert_result'] = df['CF_default'].apply(lambda x: 1 if x >= 0.6 else 0)

FUZZY

Untuk mengelompokkan nasabah ke dalam cluster lunak (tidak kaku) berdasarkan:
*  Umur
*  Penghasilan
*  Lama Kerja

Metode: Fuzzy C-Means (FCM) → tiap nasabah bisa berada di antara 2 cluster (dengan persentase keanggotaan)

Outputnya: Cluster_Fuzzy = 0 atau 1, tapi secara internal FCM tahu bahwa seseorang bisa:

60% masuk cluster risiko tinggi, 40% risiko rendah

In [7]:
# FUZZY CLUSTERING
fcm = FCM(n_clusters=2, random_state=42)
fcm.fit(df[['person_age', 'person_income', 'person_emp_length']].values)
df['fuzzy_cluster'] = fcm.predict(df[['person_age', 'person_income', 'person_emp_length']].values)

FITUR & TARGET

* X = semua kolom fitur yang dipakai untuk memprediksi.
* y = target/output, yaitu Default → 1 = gagal bayar, 0 = tidak gagal.

SCALING + SMOTE

* Scaling → Mengubah semua nilai fitur ke rentang [0, 1].
* SMOTE → Teknik oversampling: membuat salinan sintetis dari kelas minoritas.

SPLIT DATA

* 70% → X_train dan y_train → untuk melatih model.
* 30% → X_test dan y_test → untuk menguji performa model.



In [8]:
# FEATURES & TARGET
features = [
    'person_age', 'person_income', 'person_emp_length',
    'loan_to_income_ratio', 'stable_job', 'has_fixed_asset',
    'cb_person_default_on_file', 'CF_default', 'CF_nondefault', 'CF_diff',
    'expert_result', 'fuzzy_cluster'
]
X = df[features]
y = df['loan_status']

# SCALING + SMOTE
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_res, y_res = SMOTE(random_state=42).fit_resample(X_scaled, y)

# SPLIT DATA (70% TRAIN & 30% TEST)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

MODEL MACHINE LEARNING

* XGBClassifier: model gradient boosting yang sangat kuat.
* RandomForestClassifier: model ensemble dari banyak decision tree.
* LogisticRegression: model dasar
* VotingClassifier: menggabungkan ketiganya (soft voting = rata-rata probabilitas).

TRAINING & EVALUASI MODEL

Menampilkan classification report:

* Precision: ketepatan model (berapa banyak yang diprediksi gagal bayar yang benar-benar gagal).
* Recall: seberapa banyak gagal bayar yang berhasil ditangkap model.
* F1-Score: keseimbangan antara precision dan recall.
* Accuracy: total yang diprediksi benar / total data.

SHAP

Menjelaskan kontribusi tiap fitur terhadap prediksi.

In [ ]:
# DEFINE MODEL (ML)
xgb = XGBClassifier(eval_metric='logloss', random_state=42)
rf = RandomForestClassifier(random_state=42)
logreg = LogisticRegression(max_iter=1000)
voting = VotingClassifier(estimators=[('xgb', xgb), ('rf', rf), ('logreg', logreg)], voting='soft')

# TRAIN & EVALUATE
for name, model in zip(['XGBoost', 'Random Forest', 'Logistic Regression', 'Voting'], [xgb, rf, logreg, voting]):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    print(f"\n[{name} Report]")
    print(classification_report(y_test, y_pred))
    print(f"AUC {name}: {auc:.4f}")

# SHAP EXPLAINABILITY (XGBoost)
explainer = shap.Explainer(xgb, X_train, feature_names=X.columns)
shap_values = explainer(X_test[:50])
shap.summary_plot(shap_values, X_test[:50], plot_type="bar")

# FINAL INSIGHT (BASED ON MODEL)
print("""
\n=== CREDIT RISK INSIGHT ===

High Risk Profile:
- Income < 4 million
- Age < 25
- Job tenure < 1 year
- No home ownership
- Has credit default history
- High loan-to-income ratio (>60%)
- Expert system concludes default

Low Risk Profile:
- Income > 8 million
- Age > 30
- Job tenure > 5 years
- Has fixed asset (own house)
- Good credit history
- Low loan-to-income ratio (<30%)
- Supported by expert system & fuzzy clustering

""")

MENYIMPAN MODEL

In [ ]:
# SAVE MODEL (ALL MODELS COMBINED = VotingClassifier)
joblib.dump(voting, "ubmager_model.pkl", compress=3)
print("Model saved as ubmager_model.pkl")